In [7]:
pip install google.colab
!pip install -q kaggle
from google.colab import files
files.upload()

In [11]:
! mkdir ~/.kaggle

In [4]:
! cp kaggle.json ~/.kaggle/

In [5]:
! chmod 600 ~/.kaggle/kaggle.json

In [6]:
! kaggle datasets list

ref                                                         title                                               size  lastUpdated          downloadCount  
----------------------------------------------------------  -------------------------------------------------  -----  -------------------  -------------  
yamqwe/omicron-covid19-variant-daily-cases                  Omicron daily cases by country (COVID-19 variant)  432KB  2022-01-24 07:15:53          10042  
iamsouravbanerjee/analytics-industry-salaries-2022-india    Data Professionals Salary - 2022                    56KB  2022-01-28 08:47:00           1369  
nkitgupta/jigsaw-regression-based-data                      Jigsaw Regression Based Data                         3GB  2022-01-10 06:29:59            562  
prasertk/netflix-subscription-price-in-different-countries  Netflix subscription fee in different countries      3KB  2022-01-15 07:06:09           1207  
yasserh/song-popularity-dataset                             Song Popul

In [7]:
!kaggle datasets download -d new-york-city/nyc-parking-tickets


100% 2.02G/2.02G [00:19<00:00, 36.7MB/s]
100% 2.02G/2.02G [00:19<00:00, 113MB/s] 


In [8]:
!unzip nyc-parking-tickets.zip

Archive:  nyc-parking-tickets.zip
  inflating: Parking_Violations_Issued_-_Fiscal_Year_2014__August_2013___June_2014_.csv  
  inflating: Parking_Violations_Issued_-_Fiscal_Year_2015.csv  
  inflating: Parking_Violations_Issued_-_Fiscal_Year_2016.csv  
  inflating: Parking_Violations_Issued_-_Fiscal_Year_2017.csv  


In [10]:
import pandas as pd
df = pd.read_csv("/content/Parking_Violations_Issued_-_Fiscal_Year_2014__August_2013___June_2014_.csv")

<bound method NDFrame.describe of          Summons Number Plate ID Registration State  ... BIN BBL  NTA
0            1283294138  GBB9093                 NY  ... NaN NaN  NaN
1            1283294151  62416MB                 NY  ... NaN NaN  NaN
2            1283294163  78755JZ                 NY  ... NaN NaN  NaN
3            1283294175  63009MA                 NY  ... NaN NaN  NaN
4            1283294187  91648MC                 NY  ... NaN NaN  NaN
...                 ...      ...                ...  ...  ..  ..  ...
9100273      8006150199  16088MC                 NY  ... NaN NaN  NaN
9100274      8006150217  94008JW                 NY  ... NaN NaN  NaN
9100275      8006150242  FWG3729                 NY  ... NaN NaN  NaN
9100276      8006150254   193YUR                 CT  ... NaN NaN  NaN
9100277      8006150278  FPN7411                 NY  ... NaN NaN  NaN

[9100278 rows x 51 columns]>

In [11]:
df.columns

<bound method NDFrame.head of          Summons Number Plate ID Registration State  ... BIN BBL  NTA
0            1283294138  GBB9093                 NY  ... NaN NaN  NaN
1            1283294151  62416MB                 NY  ... NaN NaN  NaN
2            1283294163  78755JZ                 NY  ... NaN NaN  NaN
3            1283294175  63009MA                 NY  ... NaN NaN  NaN
4            1283294187  91648MC                 NY  ... NaN NaN  NaN
...                 ...      ...                ...  ...  ..  ..  ...
9100273      8006150199  16088MC                 NY  ... NaN NaN  NaN
9100274      8006150217  94008JW                 NY  ... NaN NaN  NaN
9100275      8006150242  FWG3729                 NY  ... NaN NaN  NaN
9100276      8006150254   193YUR                 CT  ... NaN NaN  NaN
9100277      8006150278  FPN7411                 NY  ... NaN NaN  NaN

[9100278 rows x 51 columns]>

In [13]:
%%writefile testutility.py
import logging
import os
import subprocess
import yaml
import pandas as pd
import datetime
import gc
import re

def read_config_file(filepath):
    with open(filepath, 'r') as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            logging.error(exc)


def replacer(string, char):
    pattern = char + '{2,}'
    string = re.sub(pattern, char, string)
    return string


def col_header_val(df, table_config):
    """ replace whitespaces in the column  and standardized column names """
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace('[^\w]', '_', regex=True)
    df.columns = list(map(lambda x: x.strip('_'), list(df.columns)))
    df.columns = list(map(lambda x: replacer(x, '_'), list(df.columns)))
    expected_col = list(map(lambda x: x.lower(), table_config['columns']))
    expected_col.sort()
    df.columns = list(map(lambda x: x.lower(), list(df.columns)))
    df = df.reindex(sorted(df.columns), axis=1)
    if len(df.columns) == len(expected_col) and list(expected_col) == list(df.columns):
        print("column name and column length validation passed")
        return 1
    else:
        print("column name and column length validation failed")
        mismatched_columns_file = list(set(df.columns).difference(expected_col))
        print("Following File columns are not in the YAML file", mismatched_columns_file)
        missing_YAML_file = list(set(expected_col).difference(df.columns))
        print("Following YAML columns are not in the file uploaded", missing_YAML_file)
        logging.info(f'df columns: {df.columns}')
        logging.info(f'expected columns: {expected_col}')
        return 0

Writing testutility.py


Overwriting test_utility.py

In [14]:
%%writefile file.yaml
file_type: csv
dataset_name: testfile
file_name: Parking_Violations_Issued_-_Fiscal_Year_2014__August_2013___June_2014_
table_name: edsurv
inbound_delimeter: ","
outbound_delimeter: "|"
skip_leading_rows: 1
columns:
    -Summons Number
    -Plate ID
    -Registration State
    -Plate Type
    -Issue Date
    -Violation Code
    -Vehicle Body Type
    -Vehicle Make
    -Issuing Agency
    -Street Code1
    -Street Code2
    -Street Code3
    -Vehicle Expiration Date
    -Violation Location
    -Violation Precinct
    -Issuer Precinct
    -Issuer Code
    -Issuer Command
    -Issuer Squad
    -Violation Time
    -Time First Observed
    -Violation County
    -Violation In Front Of Or Opposite
    -House Number
    -Street Name
    -Intersecting Street-Date First Observed
    -Law Section
    -Sub Division
    -Violation Legal Code
    -Days Parking In Effect    
    -From Hours In Effect
    -To Hours In Effect
    -Vehicle Color
    -Unregistered Vehicle?
    -Vehicle Year
    -Meter Number
    -Feet From Curb-Violation Post Code
    -Violation Description
    -No Standing or Stopping Violation
    -Hydrant Violation
    -Double Parking Violation
    -Latitude
    -Longitude
    -Community Board
    -Community Council 
    -Census Tract
    -BIN
    -BBL
    -NTA

Writing file.yaml


Overwriting file.yaml

In [2]:
# Read config file
import testutility as util
config_data = util.read_config_file("file.yaml")
config_data['file_type']

'csv'

In [3]:
config_data

{'columns': '-Summons Number -Plate ID -Registration State -Plate Type -Issue Date -Violation Code -Vehicle Body Type -Vehicle Make -Issuing Agency -Street Code1 -Street Code2 -Street Code3 -Vehicle Expiration Date -Violation Location -Violation Precinct -Issuer Precinct -Issuer Code -Issuer Command -Issuer Squad -Violation Time -Time First Observed -Violation County -Violation In Front Of Or Opposite -House Number -Street Name -Intersecting Street-Date First Observed -Law Section -Sub Division -Violation Legal Code -Days Parking In Effect -From Hours In Effect -To Hours In Effect -Vehicle Color -Unregistered Vehicle? -Vehicle Year -Meter Number -Feet From Curb-Violation Post Code -Violation Description -No Standing or Stopping Violation -Hydrant Violation -Double Parking Violation -Latitude -Longitude -Community Board -Community Council -Census Tract -BIN -BBL -NTA',
 'dataset_name': 'testfile',
 'file_name': 'Parking_Violations_Issued_-_Fiscal_Year_2014__August_2013___June_2014_',
 '

In [1]:
import pandas as pd
df_sample = pd.read_csv("Parking_Violations_Issued_-_Fiscal_Year_2014__August_2013___June_2014_.csv",delimiter=',')
df_sample.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (17,18,20,21,22,23,29,30,31,32,36,38,39) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Summons Number,Plate ID,Registration State,Plate Type,Issue Date,Violation Code,Vehicle Body Type,Vehicle Make,Issuing Agency,Street Code1,Street Code2,Street Code3,Vehicle Expiration Date,Violation Location,Violation Precinct,Issuer Precinct,Issuer Code,Issuer Command,Issuer Squad,Violation Time,Time First Observed,Violation County,Violation In Front Of Or Opposite,House Number,Street Name,Intersecting Street,Date First Observed,Law Section,Sub Division,Violation Legal Code,Days Parking In Effect,From Hours In Effect,To Hours In Effect,Vehicle Color,Unregistered Vehicle?,Vehicle Year,Meter Number,Feet From Curb,Violation Post Code,Violation Description,No Standing or Stopping Violation,Hydrant Violation,Double Parking Violation,Latitude,Longitude,Community Board,Community Council,Census Tract,BIN,BBL,NTA
0,1283294138,GBB9093,NY,PAS,08/04/2013,46,SUBN,AUDI,P,37250,13610,21190,20140831,33.0,33,33,921043,0033,0,0752A,NaN,NaN,F,712,W 175 ST,NaN,0.0,408.0,F1,NaN,BBBBBBB,ALL,ALL,GY,0.0,2013.0,-,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1283294151,62416MB,NY,COM,08/04/2013,46,VAN,FORD,P,37290,40404,40404,20140430,33.0,33,33,921043,0033,0,1240P,NaN,NY,O,201,W 177 ST,NaN,0.0,408.0,C,NaN,BBBBBBB,ALL,ALL,WH,0.0,2012.0,-,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1283294163,78755JZ,NY,COM,08/05/2013,46,P-U,CHEVR,P,37030,31190,13610,20140228,33.0,33,33,921043,0033,0,1243P,NaN,NY,O,520,W 163 ST,NaN,0.0,408.0,F7,NaN,BBBBBBB,ALL,ALL,NaN,0.0,0.0,-,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1283294175,63009MA,NY,COM,08/05/2013,46,VAN,FORD,P,37270,11710,12010,20141031,33.0,33,33,921043,0033,0,0232P,NaN,NY,O,517,W 176 ST,NaN,0.0,408.0,F1,NaN,BBBBBBB,ALL,ALL,WH,0.0,2010.0,-,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1283294187,91648MC,NY,COM,08/08/2013,41,TRLR,GMC,P,37240,12010,31190,0,33.0,33,33,921043,0033,0,1239P,NaN,NY,F,590,W 174 ST,NaN,0.0,408.0,E1,NaN,BBBBBBB,ALL,ALL,BR,0.0,2012.0,-,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
import pandas as pd 


data=pd.read_csv("/content/Parking_Violations_Issued_-_Fiscal_Year_2014__August_2013___June_2014_.csv",config_data['inbound_delimeter'])
print("New dataset:", data)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (17,18,20,21,22,23,29,30,31,32,36,38,39) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


New dataset:          Summons Number Plate ID Registration State  ... BIN BBL  NTA
0            1283294138  GBB9093                 NY  ... NaN NaN  NaN
1            1283294151  62416MB                 NY  ... NaN NaN  NaN
2            1283294163  78755JZ                 NY  ... NaN NaN  NaN
3            1283294175  63009MA                 NY  ... NaN NaN  NaN
4            1283294187  91648MC                 NY  ... NaN NaN  NaN
...                 ...      ...                ...  ...  ..  ..  ...
9100273      8006150199  16088MC                 NY  ... NaN NaN  NaN
9100274      8006150217  94008JW                 NY  ... NaN NaN  NaN
9100275      8006150242  FWG3729                 NY  ... NaN NaN  NaN
9100276      8006150254   193YUR                 CT  ... NaN NaN  NaN
9100277      8006150278  FPN7411                 NY  ... NaN NaN  NaN

[9100278 rows x 51 columns]


In [5]:
#validate the header of the file
util.col_header_val(data,config_data)

column name and column length validation failed
Following File columns are not in the YAML file ['street_code1', 'street_name', 'plate_type', 'bbl', 'to_hours_in_effect', 'violation_precinct', 'feet_from_curb', 'bin', 'issuer_code', 'double_parking_violation', 'meter_number', 'vehicle_year', 'violation_in_front_of_or_opposite', 'community_council', 'violation_location', 'unregistered_vehicle', 'issuing_agency', 'street_code2', 'violation_description', 'issuer_squad', 'issuer_precinct', 'plate_id', 'vehicle_body_type', 'vehicle_expiration_date', 'issuer_command', 'community_board', 'sub_division', 'street_code3', 'violation_county', 'law_section', 'intersecting_street', 'hydrant_violation', 'latitude', 'house_number', 'no_standing_or_stopping_violation', 'census_tract', 'violation_code', 'date_first_observed', 'nta', 'issue_date', 'summons_number', 'vehicle_color', 'violation_post_code', 'time_first_observed', 'vehicle_make', 'violation_legal_code', 'registration_state', 'longitude', 'd

0

In [6]:
print("columns of files are:", data.columns)
print("columns of YAML are:",config_data['columns'])

columns of files are: Index(['summons_number', 'plate_id', 'registration_state', 'plate_type',
       'issue_date', 'violation_code', 'vehicle_body_type', 'vehicle_make',
       'issuing_agency', 'street_code1', 'street_code2', 'street_code3',
       'vehicle_expiration_date', 'violation_location', 'violation_precinct',
       'issuer_precinct', 'issuer_code', 'issuer_command', 'issuer_squad',
       'violation_time', 'time_first_observed', 'violation_county',
       'violation_in_front_of_or_opposite', 'house_number', 'street_name',
       'intersecting_street', 'date_first_observed', 'law_section',
       'sub_division', 'violation_legal_code', 'days_parking_in_effect',
       'from_hours_in_effect', 'to_hours_in_effect', 'vehicle_color',
       'unregistered_vehicle', 'vehicle_year', 'meter_number',
       'feet_from_curb', 'violation_post_code', 'violation_description',
       'no_standing_or_stopping_violation', 'hydrant_violation',
       'double_parking_violation', 'latitude', '

In [7]:
if util.col_header_val(data,config_data)==0:
    print("validation failed")
else:
    print("col validation passed")

column name and column length validation failed
Following File columns are not in the YAML file ['street_code1', 'street_name', 'plate_type', 'bbl', 'to_hours_in_effect', 'violation_precinct', 'feet_from_curb', 'bin', 'issuer_code', 'double_parking_violation', 'meter_number', 'vehicle_year', 'violation_in_front_of_or_opposite', 'community_council', 'violation_location', 'unregistered_vehicle', 'issuing_agency', 'street_code2', 'violation_description', 'issuer_squad', 'issuer_precinct', 'plate_id', 'vehicle_body_type', 'vehicle_expiration_date', 'issuer_command', 'community_board', 'sub_division', 'street_code3', 'violation_county', 'law_section', 'intersecting_street', 'hydrant_violation', 'latitude', 'house_number', 'no_standing_or_stopping_violation', 'census_tract', 'violation_code', 'date_first_observed', 'nta', 'issue_date', 'summons_number', 'vehicle_color', 'violation_post_code', 'time_first_observed', 'vehicle_make', 'violation_legal_code', 'registration_state', 'longitude', 'd

In [1]:
import pandas as pd
pd.read_csv("/content/Parking_Violations_Issued_-_Fiscal_Year_2014__August_2013___June_2014_.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (17,18,20,21,22,23,29,30,31,32,36,38,39) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Summons Number,Plate ID,Registration State,Plate Type,Issue Date,Violation Code,Vehicle Body Type,Vehicle Make,Issuing Agency,Street Code1,Street Code2,Street Code3,Vehicle Expiration Date,Violation Location,Violation Precinct,Issuer Precinct,Issuer Code,Issuer Command,Issuer Squad,Violation Time,Time First Observed,Violation County,Violation In Front Of Or Opposite,House Number,Street Name,Intersecting Street,Date First Observed,Law Section,Sub Division,Violation Legal Code,Days Parking In Effect,From Hours In Effect,To Hours In Effect,Vehicle Color,Unregistered Vehicle?,Vehicle Year,Meter Number,Feet From Curb,Violation Post Code,Violation Description,No Standing or Stopping Violation,Hydrant Violation,Double Parking Violation,Latitude,Longitude,Community Board,Community Council,Census Tract,BIN,BBL,NTA
0,1283294138,GBB9093,NY,PAS,08/04/2013,46,SUBN,AUDI,P,37250,13610,21190,20140831,33.0,33,33,921043,0033,0,0752A,NaN,NaN,F,712,W 175 ST,NaN,0.0,408.0,F1,NaN,BBBBBBB,ALL,ALL,GY,0.0,2013.0,-,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1283294151,62416MB,NY,COM,08/04/2013,46,VAN,FORD,P,37290,40404,40404,20140430,33.0,33,33,921043,0033,0,1240P,NaN,NY,O,201,W 177 ST,NaN,0.0,408.0,C,NaN,BBBBBBB,ALL,ALL,WH,0.0,2012.0,-,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1283294163,78755JZ,NY,COM,08/05/2013,46,P-U,CHEVR,P,37030,31190,13610,20140228,33.0,33,33,921043,0033,0,1243P,NaN,NY,O,520,W 163 ST,NaN,0.0,408.0,F7,NaN,BBBBBBB,ALL,ALL,NaN,0.0,0.0,-,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1283294175,63009MA,NY,COM,08/05/2013,46,VAN,FORD,P,37270,11710,12010,20141031,33.0,33,33,921043,0033,0,0232P,NaN,NY,O,517,W 176 ST,NaN,0.0,408.0,F1,NaN,BBBBBBB,ALL,ALL,WH,0.0,2010.0,-,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1283294187,91648MC,NY,COM,08/08/2013,41,TRLR,GMC,P,37240,12010,31190,0,33.0,33,33,921043,0033,0,1239P,NaN,NY,F,590,W 174 ST,NaN,0.0,408.0,E1,NaN,BBBBBBB,ALL,ALL,BR,0.0,2012.0,-,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9100273,8006150199,16088MC,NY,COM,06/25/2014,38,VAN,CHEVR,T,67530,26230,84730,88888888,70.0,70,70,350442,T302,K,0346P,NaN,K,F,522,Ocean Ave,NaN,0.0,408.0,h1,NaN,Y,0900A,0700P,WHITE,NaN,2011.0,NaN,0.0,10 3,38-Failure to Display Muni Rec,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9100274,8006150217,94008JW,NY,COM,06/25/2014,38,VAN,CHEVR,T,26230,35680,67530,88888888,70.0,70,70,350442,T302,K,0414P,NaN,K,F,1913,Church Ave,NaN,0.0,408.0,h1,NaN,Y,0800A,0700P,WHITE,NaN,2007.0,NaN,0.0,10 3,38-Failure to Display Muni Rec,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9100275,8006150242,FWG3729,NY,PAS,06/25/2014,70,SUBN,FORD,T,40930,40430,48230,20140531,70.0,70,70,350442,T302,K,0448P,NaN,K,F,654,Flatbush Ave,NaN,0.0,408.0,j3,NaN,YYYYYYY,NaN,NaN,WH,NaN,1996.0,NaN,0.0,10 3,70A-Reg. Sticker Expired (NYS),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9100276,8006150254,193YUR,CT,PAS,06/25/2014,37,SUBN,CHEVR,T,40930,60830,74230,20140088,70.0,70,70,350442,T302,K,0454P,0229P,K,F,606,Flatbush Ave,NaN,0.0,408.0,h1,NaN,Y,0800A,0700P,BLACK,NaN,0.0,304-3007,0.0,10 3,37-Expired Muni Meter,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
